# Object 目的
- Understanding overview of the competition.<br>
コンペの概要を把握する。
- Understaning the train data and perform a simple visualization.<br>
学習データを理解して、簡易的なビジュアライゼーションを行う。

# 1. Overview 概要
- **the problem we'd like to solve 解きたい問題**<br>
you'll help accelerate the world’s understanding of the relationships between cell and tissue organization.<br>
細胞や組織の構造に関する理解を加速させ、人間の健康、長寿に繋げたい。
- **the problem to be solved 解くべき問題**<br>
you’ll identify and segment functional tissue units (FTUs) across five human organs. You'll build your model using a dataset of tissue section images, with the best submissions segmenting FTUs as accurately as possible.<br>
人間の組織切片画像のデータセットを使用してモデルを構築し、functional tissue units (FTUs) セグメンテーションする。
- **train data 学習データ**<br>
biopsy slides from several different organs<br>
複数の異なる臓器の生検スライド<br>
The training dataset consists of data from public HPA data, the public test set is a combination of private HPA data and HuBMAP data, and the private test set contains only HuBMAP data.<br>
学習データセットは公開されているHPAのデータ<br>
公開テストセットは非公開のHPAデータとHuBMAPデータの組み合わせ<br>
非公開テストセットはHuBMAPデータのみ
- **Supervised ML Evaluation 評価手法**<br>
mean Dice coefficient is familiar in image segmentation.<br>
画像セグメンテーションではおなじみのDice係数<br>
The Dice coefficient can be used to compare the pixel-wise agreement between a predicted segmentation and its corresponding ground truth.<br>
2つの集合の平均要素数と共通要素数の割合で、1に近づくほど高スコア<br>
$$
\frac{2*|X\cap Y|}{|X|+|Y|}\\
$$
$X$:predicted set of pixels 予測したピクセルの集合<br>
$Y$:ground truth 正解ピクセルの集合<br>

# 2. Understanding Train Data 学習データの理解

In [ ]:
import os
import cv2
import tifffile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_train = pd.read_csv('../input/hubmap-organ-segmentation/train.csv')
df_test = pd.read_csv('../input/hubmap-organ-segmentation/test.csv')
df_train.sample(5)

In [ ]:
df_train.describe(include='all')

In [ ]:
df_test

In [ ]:
sns.countplot(x='sex',data=df_train)

Male have about twice as much data.<br>
男性の方が約2倍データが多い。

In [ ]:
sns.countplot(x='organ', hue='sex',data=df_train)

- The prostate is in male data only.<br>
前立腺は男性のみ存在する臓器
- The kidney common in male data.<br>
肝臓は男性データに多い
- Almost the same number of the spleen, the lung organ and the largeintestine for male and female.<br>
脾臓、肺臓、大腸は、男女でだいたい同じ数。

In [ ]:
fig = plt.figure(figsize=(8,3))
ax1 = fig.add_subplot(1, 2, 1)
ax1 = df_train['data_source'].value_counts().plot.bar()
ax1.set_title('train')
ax2 = fig.add_subplot(1, 2, 2)
ax2 = df_test['data_source'].value_counts().plot.bar()
ax2.set_title('test')
fig.show()

All train data are HPA.<br>
学習データは全てHPA。

In [ ]:
sns.jointplot(x='img_height',y='img_width',data=df_train)
plt.show()

The train image is mostly square images.<br>
ほとんど正方形の画像、ほとんどが3000×3000の画像サイズだが、例外もある。

In [ ]:
sns.countplot(x='pixel_size',data=df_train)
plt.show()

In [ ]:
sns.countplot(x='tissue_thickness',data=df_train)
plt.show()

In [ ]:
(ax1, ax2), = pd.crosstab(pd.cut(df_train['age'], range(0, 91, 5), right=False), df_train['sex']).reindex(columns=['Male', 'Female']).plot.barh(subplots=True, layout=(1, 2), sharex=False)
ax1.invert_xaxis()
ax2.set_yticklabels([])
ax1.get_legend().remove()
ax2.get_legend().remove()
plt.show()

In [ ]:
df_train['rle']

This encoding method is often used in image segmentation.<br>
For example, "1 3 10 5" means that pixels 1, 2, 3, 10, 11, 12, 13, and 14 are included in the mask.<br>
The same encoding should be used for submission.<br>
マスク情報はrleエンコーディングされている。画像セグメンテーションでは、よく用いられるエンコーディング方式。<br>
例えば、「1 3 10 5」は、ピクセル1、2、3、10、11、12、13、14がマスクに含まれることを意味する。<br>
提出時も同様にエンコーディングする必要がある。

# 3. Visualization 可視化

In [ ]:
#https://www.kaggle.com/code/pestipeti/decoding-rle-masks/notebook
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle2mask(mask_rle, shape=(3000,3000)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
# 各organごとに、ランダムに1枚選んで表示
organs = np.unique(df_train['organ'])
fig, ax = plt.subplots(3,len(organs),figsize=(20,12))
for i in range(len(organs)):
    df_organ = df_train[df_train['organ']==organs[i]].reset_index()
    idx = np.random.randint(df_organ.shape[0])
    image = plt.imread(f"../input/hubmap-organ-segmentation/train_images/{df_organ.id[idx]}.tiff")
    mask = rle2mask(df_organ.rle[idx],shape=(df_organ.img_height[idx],df_organ.img_width[idx]))
    ax[0,i].imshow(image)
    ax[0,i].set_title(df_organ.organ[idx])
    ax[0,i].axis("off")
    ax[1,i].imshow(mask,alpha=0.3,cmap='gray')
    ax[1,i].axis("off")
    ax[2,i].imshow(image)
    ax[2,i].imshow(mask,alpha=0.3,cmap='gray')
    ax[2,i].axis("off")

# 4. Submittion File 提出ファイル

In [ ]:
sub_df=pd.read_csv('../input/hubmap-organ-segmentation/sample_submission.csv')
sub_df

The submit csv include the image id and the rle-encoded segmentation.<br>
画像idとrleエンコーディングしたセグメンテーション情報を提出

In [ ]:
# 提出する際は、indexをつけないように出力
sub_df.to_csv('submission.csv',index=False)